In [1]:
import torch
inputs = torch.tensor([
   [0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

# For 'Journey'

In [2]:
input_2 = inputs[1]
input_2

tensor([0.5500, 0.8700, 0.6600])

In [3]:
d_in = inputs.shape[1]  # embedding size (3)
d_out = 2

In [4]:
# generate (3*2) size matrix for query, key, value

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)

In [5]:
W_query

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])

In [6]:
# single input (row 2) -> transform
query_2 = input_2 @ W_query
key_2 = input_2 @ W_key
value_2 = input_2 @ W_value

In [7]:
query_2

tensor([0.4306, 1.4551])

In [8]:
value_2

tensor([0.3951, 1.0037])

In [9]:
key_2

tensor([0.4433, 1.1419])

### (w22) =  For 'Journey' --> key_2  

In [10]:
attention_score_22 = query_2.dot(key_2)
attention_score_22

tensor(1.8524)

### (w21, w22, w23, ..., w2T) =  For 'Journey' --> keys 

In [11]:
keys = inputs @ W_key
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])

In [12]:
attention_score_2 = query_2 @ keys.T
attention_score_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

In [13]:
# normalize
d_k = keys.shape[-1]
attention_weights_2 = torch.softmax(attention_score_2 / d_k**0.5, dim=-1)
attention_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [14]:
## context vector weighted sum of attention_wight and values

values = inputs @ W_value
values

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])

In [15]:
context_vector = attention_weights_2 @ values
context_vector

tensor([0.3061, 0.8210])

# For all query

In [16]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [17]:
queries = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value

In [18]:
queries

tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]])

In [19]:
attention_scores = queries @ keys.T
attention_scores

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])

In [20]:
d_k = keys.shape[-1]
print(d_k)

attention_weights = torch.softmax(attention_scores / d_k**0.5, dim=-1)
attention_weights

2


tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])

In [21]:
context_vectors = attention_weights @ values
context_vectors

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])

# Implement Class

In [25]:
import torch.nn as nn

class SelfAttentionOwnClass(nn.Module):
   def __init__(self, d_in, d_out):
      super().__init__()
      self.W_query = nn.Parameter(torch.rand(d_in,d_out))
      self.W_key = nn.Parameter(torch.rand(d_in,d_out))
      self.W_value = nn.Parameter(torch.rand(d_in,d_out))
      
   def forward(self,x):
      queries = x @ self.W_query
      keys = x @ self.W_key
      values = x @ self.W_value
      
      attention_scores = queries @ keys.T
      d_k = keys.shape[-1]
      attention_weights = torch.softmax(attention_scores/ d_k**0.5, dim=-1)
      context_vector = attention_weights @ values
      return context_vector

In [26]:
torch.manual_seed(123)
sA = SelfAttentionOwnClass(d_in, d_out)

In [27]:
sA(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)